In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import gspread
import pandas as pd


In [ ]:
pd.set_option('display.max_rows', None)


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/LOMBA/fortex/data/Copy of 2023_us - 2023_us.csv.csv")

In [ ]:
df.info()

In [ ]:
# Initialize lists to store the results
missing_percentage = []
medians = []
means = []
unique_values = []

# Loop through each column
for col in df.columns:
    # Calculate percentage of missing values
    missing_percentage.append(df[col].isna().mean() * 100)

    # Check if the column is numeric before calculating median and mean
    if pd.api.types.is_numeric_dtype(df[col]):
        medians.append(df[col].median())  # Calculate median
        means.append(df[col].mean())  # Calculate mean
    else:
        medians.append(None)  # Set as None for non-numeric columns
        means.append(None)  # Set as None for non-numeric columns

    # Calculate number of unique values for every column
    unique_values.append(df[col].nunique())

# Create a summary DataFrame with the results
summary_df = pd.DataFrame({
    'Column': df.columns,
    'Missing Percentage (%)': missing_percentage,
    'Median': medians,
    'Mean': means,
    'Unique Values': unique_values
})

# Display the summary DataFrame
print(summary_df)

In [ ]:
summary_df_sorted = summary_df.sort_values(by='Missing Percentage (%)', ascending=False)


In [ ]:
summary_df_sorted

In [ ]:
filtered_columns = summary_df[summary_df['Missing Percentage (%)'] < 70]


In [ ]:
filtered_columns.sort_values(by='Missing Percentage (%)', ascending=False)

In [ ]:
filtered_columns = summary_df[summary_df['Missing Percentage (%)'] < 70]

# Initialize lists for classification
numeric_columns = []
categorical_columns = []
other_columns = []

# Classify columns into numeric, categorical, and other categories
for col in filtered_columns['Column']:
    if pd.api.types.is_numeric_dtype(df[col]):
        numeric_columns.append(col)
    elif pd.api.types.is_categorical_dtype(df[col]) or df[col].dtype == 'object':
        categorical_columns.append(col)
    else:
        other_columns.append(col)

In [ ]:
len(categorical_columns)

In [ ]:
len(numeric_columns)

In [ ]:
len(other_columns)

In [ ]:
79+26

In [ ]:
# List of columns to exclude
columns_to_exclude = [
    "1. YEAR", "2. TRIFD", "3. FRS ID", "4. FACILITY NAME", "5. STREET ADDRESS",
    "6. CITY", "7. COUNTY", "8. ST", "9. ZIP", "10. BIA", "11. TRIBE", "12. LATITUDE",
    "13. LONGITUDE", "14. HORIZONTAL DATUM", "15. PARENT CO NAME", "16. PARENT CO DB NUM",
    "17. STANDARD PARENT CO NAME", "18. FOREIGN PARENT CO NAME", "19. FOREIGN PARENT CO DB NUM",
    "20. STANDARD FOREIGN PARENT CO NAME", "23. INDUSTRY SECTOR", "24. PRIMARY SIC",
    "25. SIC 2", "26. SIC 3", "27. SIC 4", "28. SIC 5", "29. SIC 6", "30. PRIMARY NAICS",
    "31. NAICS 2", "32. NAICS 3", "33. NAICS 4", "34. NAICS 5", "35. NAICS 6",
    "36. DOC_CTRL_NUM", "39. TRI CHEMICAL/COMPOUND ID", "40. CAS#", "41. SRS ID"
]

# Exclude those columns from the DataFrame
filtered_columns = filtered_columns[~filtered_columns['Column'].isin(columns_to_exclude)]




In [ ]:
filtered_columns

In [ ]:
filtered_columns.sort_values(by='Missing Percentage (%)', ascending=False)

In [ ]:
filtered_columns[filtered_columns['Unique Values']==1]['Column'].values

In [ ]:
# Get the columns to remove
columns_to_remove = filtered_columns[filtered_columns['Unique Values'] == 1]['Column'].values

# Filter the `filtered_columns` DataFrame to exclude rows where the 'Column' is in `columns_to_remove`
filtered_columns = filtered_columns[~filtered_columns['Column'].isin(columns_to_remove)]


In [ ]:

filtered_columns

# Filter by big relation with target


In [ ]:
df_=df.copy()
df_=df_[filtered_columns['Column'].values]

In [ ]:
df_

In [ ]:
# Initialize lists to store the results
missing_percentage = []
medians = []
means = []
unique_values = []

# Loop through each column
for col in df_.columns:
    # Calculate percentage of missing values
    missing_percentage.append(df_[col].isna().mean() * 100)

    # Check if the column is numeric before calculating median and mean
    if pd.api.types.is_numeric_dtype(df_[col]):
        medians.append(df_[col].median())  # Calculate median
        means.append(df_[col].mean())  # Calculate mean
    else:
        medians.append(None)  # Set as None for non-numeric columns
        means.append(None)  # Set as None for non-numeric columns

    # Calculate number of unique values for every column
    unique_values.append(df_[col].nunique())

# Create a summary DataFrame with the results
summary_df_ = pd.DataFrame({
    'Column': df_.columns,
    'Missing Percentage (%)': missing_percentage,
    'Median': medians,
    'Mean': means,
    'Unique Values': unique_values
})

# Display the summary DataFrame
print(summary_df_)

In [ ]:
summary_df_ = summary_df_.sort_values(by="Missing Percentage (%)", ascending=False)

In [ ]:
summary_df_

In [ ]:
target='119. PRODUCTION WSTE (8.1-8.7)'
df_numeric = df_.select_dtypes(include=['number'])

# Calculate the correlation with the target column
correlation_with_target = df_numeric.corr()[target].sort_values(ascending=False)

In [ ]:
correlation_with_target = df_numeric.corr()[target].sort_values(ascending=False)

# Filter out columns with less than 1% correlation (both positive and negative)
columns_to_keep = correlation_with_target[abs(correlation_with_target) >= 0.01].index

# Filter the original DataFrame to keep only the selected columns
df_filtered = df_numeric[columns_to_keep]

In [ ]:
df_['121. PROD_RATIO_OR_ ACTIVITY']=df_['121. PROD_RATIO_OR_ ACTIVITY'].fillna(df_['121. PROD_RATIO_OR_ ACTIVITY'].mode()[0])

In [ ]:
df_['122. 8.9 - PRODUCTION RATIO']

In [ ]:
df_=df_filtered.copy()

In [ ]:
pd.set_option('display.max_rows', None)


In [ ]:
from sklearn.impute import KNNImputer

# Separate the numeric columns from the non-numeric columns
numeric_cols = df_.select_dtypes(include=['number']).columns
non_numeric_cols = df_.select_dtypes(exclude=['number']).columns

# Initialize the KNNImputer with the number of neighbors
knn_imputer = KNNImputer(n_neighbors=5)

# Apply KNN imputation to the numeric columns
df_numeric_imputed = pd.DataFrame(knn_imputer.fit_transform(df_[numeric_cols]), columns=numeric_cols)

# Combine the imputed numeric columns with the non-numeric columns
df_imputed = pd.concat([df_numeric_imputed, df_[non_numeric_cols]], axis=1)

# Optionally, display the imputed DataFrame
df_imputed.head()


In [ ]:
# Initialize lists to store the results
missing_percentage = []
medians = []
means = []
unique_values = []
df_=df_imputed.copy()
# Loop through each column
for col in df_.columns:
    # Calculate percentage of missing values
    missing_percentage.append(df_[col].isna().mean() * 100)

    # Check if the column is numeric before calculating median and mean
    if pd.api.types.is_numeric_dtype(df_[col]):
        medians.append(df_[col].median())  # Calculate median
        means.append(df_[col].mean())  # Calculate mean
    else:
        medians.append(None)  # Set as None for non-numeric columns
        means.append(None)  # Set as None for non-numeric columns

    # Calculate number of unique values for every column
    unique_values.append(df_[col].nunique())

# Create a summary DataFrame with the results
summary_df_ = pd.DataFrame({
    'Column': df_.columns,
    'Missing Percentage (%)': missing_percentage,
    'Median': medians,
    'Mean': means,
    'Unique Values': unique_values
})

# Display the summary DataFrame
print(summary_df_)

In [ ]:
summary_df_ = summary_df_.sort_values(by="Missing Percentage (%)", ascending=False)

In [ ]:
summary_df_

In [ ]:
df_.to_csv("/content/drive/MyDrive/LOMBA/fortex/data/fortex_final_data.csv",index=False)

In [ ]:
categorical_cols = df_.select_dtypes(exclude=['number']).columns
df_imputed = df_[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]), axis=0)


In [ ]:
df_imputed.isnull().sum()

In [ ]:
df_imputed.head()

In [ ]:
df_final=df_.copy()

In [ ]:
df_final[df_imputed.columns]=df_imputed

In [ ]:
from sklearn.impute import SimpleImputer

# Check for columns with missing values
missing_columns = df_final.columns[df_final.isnull().any()]
print("Columns with missing values:", missing_columns)



In [ ]:
df_final=df_final.drop(['41. SRS ID'],axis=1)

In [ ]:
df_final=df_final.drop(['2. TRIFD'],axis=1)

In [ ]:
df_final['122. 8.9 - PRODUCTION RATIO']=df['122. 8.9 - PRODUCTION RATIO'].fillna(df['122. 8.9 - PRODUCTION RATIO'].median())

In [ ]:
df_final.to_csv("/content/drive/MyDrive/LOMBA/fortex/data/final_cleaned_data2.csv",index=False)

In [ ]:
df_final.isnull().sum()

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/LOMBA/fortex/data/final_cleaned_data.csv")

In [ ]:
df.head()

In [ ]:
# List of features you want to recover
features_to_recover =  [
    '51. 5.1 - FUGITIVE AIR', '52. 5.2 - STACK AIR', '53. 5.3 - WATER', '54. 5.4 - UNDERGROUND',
    '55. 5.4.1 - UNDERGROUND CL I', '56. 5.4.2 - UNDERGROUND C II-V', '57. 5.5.1 - LANDFILLS',
    '58. 5.5.1A - RCRA C LANDFILL', '59. 5.5.1B - OTHER LANDFILLS', '60. 5.5.2 - LAND TREATMENT',
    '61. 5.5.3 - SURFACE IMPNDMNT', '62. 5.5.3A - RCRA SURFACE IM', '63. 5.5.3B - OTHER SURFACE I',
    '64. 5.5.4 - OTHER DISPOSAL', '65. ON-SITE RELEASE TOTAL', '66. 6.1 - POTW - TRNS RLSE',
    '67. 6.1 - POTW - TRNS TRT', '68. POTW - TOTAL TRANSFERS', '69. 6.2 - M10', '70. 6.2 - M41',
    '71. 6.2 - M62', '72. 6.2 - M40 METAL', '73. 6.2 - M61 METAL', '74. 6.2 - M71', '75. 6.2 - M81',
    '76. 6.2 - M82', '77. 6.2 - M72', '78. 6.2 - M63', '79. 6.2 - M66', '80. 6.2 - M67', '81. 6.2 - M64',
    '82. 6.2 - M65', '83. 6.2 - M73', '84. 6.2 - M79', '85. 6.2 - M90', '86. 6.2 - M94', '87. 6.2 - M99',
    '88. OFF-SITE RELEASE TOTAL', '89. 6.2 - M20', '90. 6.2 - M24', '91. 6.2 - M26', '92. 6.2 - M28',
    '93. 6.2 - M93', '94. OFF-SITE RECYCLED TOTAL', '95. 6.2 - M56', '96. 6.2 - M92', '97. OFF-SITE ENERGY RECOVERY T',
    '98. 6.2 - M40 NON-METAL', '99. 6.2 - M50', '100. 6.2 - M54', '101. 6.2 - M61 NON-METAL', '102. 6.2 - M69',
    '103. 6.2 - M95', '104. OFF-SITE TREATED TOTAL', '105. 6.2 - UNCLASSIFIED'
]

# Check if all the columns are in 'df'
columns_found = [col for col in features_to_recover if col in df.columns]
columns_not_found = [col for col in features_to_recover if col not in df.columns]

# If there are missing columns, print them
if columns_not_found:
    print(f"Columns not found in df: {columns_not_found}")

# Recover the columns from 'df' and add them to 'df_final'
df_final[columns_found] = df[columns_found]

# Verify if the columns have been added
print(f"Columns added to df_final: {columns_found}")


In [ ]:
df_final

In [ ]:
df_final=df_no_outliers.copy()

In [ ]:
df_final=df_.copy()

In [ ]:
pip install category-encoders


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Step 1: Convert all categorical columns to string type
categorical_cols = df_final.select_dtypes(include=['object']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Apply Label Encoding to all categorical columns
label_encoder = LabelEncoder()

for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Step 4: Apply Standard Scaling to the features
scaler = StandardScaler()

# Only scale the numeric columns (exclude the target and categorical columns)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Step 1: Convert all categorical columns to string type
# Convert all categorical columns to string to ensure consistency for encoding
categorical_cols = df_final.select_dtypes(include=['object']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

print("Categorical columns after conversion:", categorical_cols)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Step 4: Define a function to encode categorical columns
# def encode_categorical_columns(X_train, X_test, categorical_cols):
#     label_encoders = {}

#     # Apply LabelEncoder to each categorical column
#     for col in categorical_cols:
#         le = LabelEncoder()
#         le.fit(X_train[col])  # Fit encoder only on training data
#         X_train[col] = le.transform(X_train[col])

#         # Apply the same encoder to the test set, ignoring unknown labels
#         X_test[col] = X_test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)  # Map unseen labels to -1

#         label_encoders[col] = le  # Store the encoder for future use

#     return X_train, X_test, label_encoders

# # Step 5: Encode categorical columns in both training and test sets
# X_train, X_test, label_encoders = encode_categorical_columns(X_train, X_test, categorical_cols)

# Step 6: Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
X

In [ ]:
df_final['119. PRODUCTION WSTE (8.1-8.7)'].mean()

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Assuming y_test and y_pred are already defined (true values and predictions)
mse = mean_squared_error(y_test, y_pred)  # Calculate Mean Squared Error
rmse = np.sqrt(mse)  # Calculate Root Mean Squared Error

rmse


In [ ]:
rmse = np.sqrt(90203262568691.48)
print(f"Root Mean Squared Error (RMSE): {rmse}")


In [ ]:
df_final

In [ ]:
import pandas as pd

def detect_outliers(df):
    """
    Function to detect outliers in all numeric columns of a dataframe using the IQR method.
    The function returns the indices of outliers.
    """
    outlier_indices = set()  # Using a set to avoid duplicate indices

    # Loop through each numeric column in the dataframe
    for col in df.select_dtypes(include=['number']).columns:
        # Check if the column has at least two unique values (avoid constant columns)
        if df[col].nunique() > 1:
            # Drop rows with NaN values in the column (or use fillna if preferred)
            df_col = df[col].dropna()

            # Check if the column is not empty after dropping NaNs
            if len(df_col) == 0:
                continue  # Skip empty columns

            Q1 = df_col.quantile(0.25)  # First quartile (25%)
            Q3 = df_col.quantile(0.75)  # Third quartile (75%)
            IQR = Q3 - Q1  # Interquartile range

            # Handle division by zero if IQR is zero
            if IQR == 0:
                continue  # No outliers if IQR is zero

            # Define the lower and upper bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Detect outliers (values outside the bounds)
            outlier = df[col][(df[col] < lower_bound) | (df[col] > upper_bound)]

            # Add the indices of the outliers to the set
            outlier_indices.update(outlier.index)

            # Print the number of outliers in the column
            print(f"Number of outliers in {col}: {outlier.count()}")

    return outlier_indices

# Example usage:
# Assuming 'df_final' is your DataFrame with data
outliers_indices = detect_outliers(df_final)

# Now you can filter out the outliers from the DataFrame
df_no_outliers = df_final.drop(index=outliers_indices)

print(f"Filtered DataFrame without outliers: \n{df_no_outliers}")


In [ ]:
df_no_outliers.to_csv("/content/drive/MyDrive/LOMBA/fortex/data/no_outliers.csv",index=False)

In [ ]:
df_final['119. PRODUCTION WSTE (8.1-8.7)'].mean()

In [ ]:
# List of categorical columns to check unique values
categorical_columns = ['4. FACILITY NAME', '5. STREET ADDRESS', '6. CITY', '7. COUNTY',
                       '8. ST', '14. HORIZONTAL DATUM', '15. PARENT CO NAME', '16. PARENT CO DB NUM',
                       '17. STANDARD PARENT CO NAME', '21. FEDERAL FACILITY', '23. INDUSTRY SECTOR',
                       '37. CHEMICAL', '38. ELEMENTAL METAL INCLUDED', '39. TRI CHEMICAL/COMPOUND ID',
                       '40. CAS#', '42. CLEAN AIR ACT CHEMICAL', '43. CLASSIFICATION', '44. METAL',
                       '45. METAL CATEGORY', '46. CARCINOGEN', '47. PBT', '48. PFAS', '49. FORM TYPE',
                       '50. UNIT OF MEASURE', '121. PROD_RATIO_OR_ ACTIVITY']

# Print unique values for each categorical column
for col in categorical_columns:
    print(f"Unique values in '{col}':\n", df_final[col].unique(), "\n")


In [ ]:
# List of categorical columns to check unique values
categorical_columns = ['4. FACILITY NAME', '5. STREET ADDRESS', '6. CITY', '7. COUNTY',
                       '8. ST', '14. HORIZONTAL DATUM', '15. PARENT CO NAME', '16. PARENT CO DB NUM',
                       '17. STANDARD PARENT CO NAME', '21. FEDERAL FACILITY', '23. INDUSTRY SECTOR',
                       '37. CHEMICAL', '38. ELEMENTAL METAL INCLUDED', '39. TRI CHEMICAL/COMPOUND ID',
                       '40. CAS#', '42. CLEAN AIR ACT CHEMICAL', '43. CLASSIFICATION', '44. METAL',
                       '45. METAL CATEGORY', '46. CARCINOGEN', '47. PBT', '48. PFAS', '49. FORM TYPE',
                       '50. UNIT OF MEASURE', '121. PROD_RATIO_OR_ ACTIVITY']

# Print unique values for each categorical column
for col in categorical_columns:
    print(f"Unique values in '{col}':\n", df[col].unique(), "\n")


# FE

In [5]:
df=pd.read_csv("/content/drive/MyDrive/LOMBA/fortex/data/fortex_final_data.csv")

In [ ]:
df.columns

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

# 1. Feature Grouping: Define key groups related to waste and emissions.
waste_columns = ['59. 5.5.1B - OTHER LANDFILLS',
                 '60. 5.5.2 - LAND TREATMENT',
                 '62. 5.5.3A - RCRA SURFACE IM',
                 '63. 5.5.3B - OTHER SURFACE I',
                 '64. 5.5.4 - OTHER DISPOSAL',
                 '58. 5.5.1A - RCRA C LANDFILL']

emission_columns = ['51. 5.1 - FUGITIVE AIR', '52. 5.2 - STACK AIR', '53. 5.3 - WATER']

# 2. Creating composite features: Sum emissions, waste, and other groups
df['Total Waste'] = df[waste_columns].sum(axis=1, skipna=True)
df['Total Emissions'] = df[emission_columns].sum(axis=1, skipna=True)

# 3. Handling Chemical and Risk Features: Create a "Risk Factor"
df['Risk Factor'] = np.where(df['46. CARCINOGEN'] == 'Yes', 1, 0) + \
                    np.where(df['47. PBT'] == 'Yes', 1, 0) + \
                    np.where(df['48. PFAS'] == 'Yes', 1, 0)

# 4. Waste Disposal Method: Combine on-site vs off-site releases
df[' +'] = df['109. 8.1A - ON-SITE CONTAINED'] + df['110. 8.1B - ON-SITE OTHER']
df['Off-Site Waste Disposal'] = df['111. 8.1C - OFF-SITE CONTAIN'] + df['112. 8.1D - OFF-SITE OTHER R']

# 5. Create a ratio of on-site vs off-site waste disposal
# df['On-Site to Off-Site Ratio'] = df['On-Site Waste Disposal'] / (df['Off-Site Waste Disposal'] + 1)  # Adding 1 to avoid division by 0

# 6. Interaction between Total Emissions and Waste: Create a feature to indicate the relation between emissions and waste
df['Emissions to Waste Ratio'] = df['Total Emissions'] / (df['Total Waste'] + 1)

In [12]:
# df['Waste Disposal by Industry'] = df['22. INDUSTRY SECTOR CODE'].astype(str) + " " + df['On-Site Waste Disposal'].astype(str)

# Ratio between Emissions and Waste per Chemical Category (Carcinogen, PBT, PFAS)
df['Emissions to Waste per Carcinogen'] = df['Total Emissions'] / (df['Total Waste'] + 1) * np.where(df['46. CARCINOGEN'] == 'Yes', 1, 0)
df['Emissions to Waste per PBT'] = df['Total Emissions'] / (df['Total Waste'] + 1) * np.where(df['47. PBT'] == 'Yes', 1, 0)
df['Emissions to Waste per PFAS'] = df['Total Emissions'] / (df['Total Waste'] + 1) * np.where(df['48. PFAS'] == 'Yes', 1, 0)

# Metal Categories Interaction
df['Metal Waste Disposal'] = np.where(df['44. METAL'] == 'Yes', df['Total Waste'], 0)
df['Non-Metal Waste Disposal'] = np.where(df['44. METAL'] != 'Yes', df['Total Waste'], 0)

# Additional Waste Disposal Types (e.g., surface impoundments, landfills)
df['Landfill to Surface Ratio'] = df['59. 5.5.1B - OTHER LANDFILLS'] / (df['62. 5.5.3A - RCRA SURFACE IM'] + 1)

# # Ratios and Aggregations
# df['Total Waste to Facility Ratio'] = df['Total Waste'] / (df['22. INDUSTRY SECTOR CODE'].astype(str).map(df['22. INDUSTRY SECTOR CODE'].value_counts()) + 1)
# df['Total Emissions to Facility Ratio'] = df['Total Emissions'] / (df['22. INDUSTRY SECTOR CODE'].astype(str).map(df['22. INDUSTRY SECTOR CODE'].value_counts()) + 1)

In [13]:
# 1. Aggregated Release Features (e.g., POTW and Other Releases)
df['Total POTW Releases'] = df['66. 6.1 - POTW - TRNS RLSE'] + df['67. 6.1 - POTW - TRNS TRT'] + df['68. POTW - TOTAL TRANSFERS']
df['Total Off-Site Releases'] = df['88. OFF-SITE RELEASE TOTAL'] + df['94. OFF-SITE RECYCLED TOTAL'] + df['97. OFF-SITE ENERGY RECOVERY T'] + df['104. OFF-SITE TREATED TOTAL']
df['Total Releases'] = df['Total POTW Releases'] + df['Total Off-Site Releases'] + df['107. TOTAL RELEASES']

# 2. Energy Recovery and Recycling Ratios
df['Energy Recovery to Recycling'] = (df['113. 8.2 - ENERGY RECOVER ON'] + df['114. 8.3 - ENERGY RECOVER OF']) / (df['115. 8.4 - RECYCLING ON SITE'] + df['116. 8.5 - RECYCLING OFF SIT'] + 1)

# 3. Treatment Features (on-site vs off-site)
df['On-Site Treatment Ratio'] = (df['117. 8.6 - TREATMENT ON SITE']) / (df['118. 8.7 - TREATMENT OFF SITE'] + 1)


# 5. Categorical Feature Encoding
df['Chemical'] = np.where(df['37. CHEMICAL'] == 'Yes', 1, 0)
df['Elemental Metal'] = np.where(df['38. ELEMENTAL METAL INCLUDED'] == 'Yes', 1, 0)
df['Clean Air Act Chemical'] = np.where(df['42. CLEAN AIR ACT CHEMICAL'] == 'Yes', 1, 0)

# 6. Ratio Features: Energy Recovery to Total Releases, Total Transfers to Waste
df['Energy Recovery to Total Releases'] = (df['113. 8.2 - ENERGY RECOVER ON'] + df['114. 8.3 - ENERGY RECOVER OF']) / (df['Total Releases'] + 1)
df['Total Transfers to Waste'] = (df['68. POTW - TOTAL TRANSFERS'] + df['106. 6.2 - TOTAL TRANSFER']) / (df['119. PRODUCTION WSTE (8.1-8.7)'] + 1)


In [14]:
df['Total M-Series'] = df[['69. 6.2 - M10', '70. 6.2 - M41', '71. 6.2 - M62', '75. 6.2 - M81',
                            '76. 6.2 - M82', '79. 6.2 - M66', '80. 6.2 - M67', '81. 6.2 - M64',
                            '82. 6.2 - M65', '83. 6.2 - M73', '84. 6.2 - M79', '85. 6.2 - M90',
                            '86. 6.2 - M94', '87. 6.2 - M99', '89. 6.2 - M20', '90. 6.2 - M24',
                            '91. 6.2 - M26', '92. 6.2 - M28', '93. 6.2 - M93', '95. 6.2 - M56',
                            '96. 6.2 - M92', '98. 6.2 - M40 NON-METAL', '99. 6.2 - M50',
                            '100. 6.2 - M54', '101. 6.2 - M61 NON-METAL', '102. 6.2 - M69',
                            '103. 6.2 - M95']].sum(axis=1)

In [15]:
df['Average M-Series'] = df[['69. 6.2 - M10', '70. 6.2 - M41', '71. 6.2 - M62', '75. 6.2 - M81',
                            '76. 6.2 - M82', '79. 6.2 - M66', '80. 6.2 - M67', '81. 6.2 - M64',
                            '82. 6.2 - M65', '83. 6.2 - M73', '84. 6.2 - M79', '85. 6.2 - M90',
                            '86. 6.2 - M94', '87. 6.2 - M99', '89. 6.2 - M20', '90. 6.2 - M24',
                            '91. 6.2 - M26', '92. 6.2 - M28', '93. 6.2 - M93', '95. 6.2 - M56',
                            '96. 6.2 - M92', '98. 6.2 - M40 NON-METAL', '99. 6.2 - M50',
                            '100. 6.2 - M54', '101. 6.2 - M61 NON-METAL', '102. 6.2 - M69',
                            '103. 6.2 - M95']].mean(axis=1)

In [16]:
# # 2. Ratios Between M-series values (to identify patterns or relationships)
# df['M10_to_M41'] = df['69. 6.2 - M10'] / (df['70. 6.2 - M41'] + 1)

# # 3. Categorization of Non-Metal Features
# df['M40_NON_METAL'] = np.where(df['98. 6.2 - M40 NON-METAL'] > 0, 1, 0)
# df['M61_NON_METAL'] = np.where(df['101. 6.2 - M61 NON-METAL'] > 0, 1, 0)

# 4. Off-Site Release and Treatment Aggregation
df['Total Off-Site Releases'] = df[['88. OFF-SITE RELEASE TOTAL', '94. OFF-SITE RECYCLED TOTAL',
                                    '97. OFF-SITE ENERGY RECOVERY T', '104. OFF-SITE TREATED TOTAL']].sum(axis=1)
df['Total Off-Site Treatment'] = df[['109. 8.1A - ON-SITE CONTAINED', '110. 8.1B - ON-SITE OTHER',
                                     '111. 8.1C - OFF-SITE CONTAIN', '112. 8.1D - OFF-SITE OTHER R']].sum(axis=1)

# 5. Production Ratio Feature (Interaction)
df['Production_to_Off_Site_Treatment'] = df['119. PRODUCTION WSTE (8.1-8.7)'] / (df['Total Off-Site Treatment'] + 1)

In [17]:

m_series_columns = ['69. 6.2 - M10', '70. 6.2 - M41', '71. 6.2 - M62', '75. 6.2 - M81',
                    '76. 6.2 - M82', '79. 6.2 - M66', '80. 6.2 - M67', '81. 6.2 - M64',
                    '82. 6.2 - M65', '83. 6.2 - M73', '84. 6.2 - M79', '85. 6.2 - M90',
                    '86. 6.2 - M94', '87. 6.2 - M99', '89. 6.2 - M20', '90. 6.2 - M24',
                    '91. 6.2 - M26', '92. 6.2 - M28', '93. 6.2 - M93', '95. 6.2 - M56',
                    '96. 6.2 - M92', '98. 6.2 - M40 NON-METAL', '99. 6.2 - M50',
                    '100. 6.2 - M54', '101. 6.2 - M61 NON-METAL', '102. 6.2 - M69',
                    '103. 6.2 - M95']
def flag_zeros(df, columns):
    for col in columns:
        df[f'{col}_zero_flag'] = np.where(df[col] == 0, 1, 0)
    return df

df = flag_zeros(df, m_series_columns)

In [18]:
def flag_high_values(df, columns, threshold=1000):
    for col in columns:
        df[f'{col}_high_flag'] = np.where(df[col] > threshold, 1, 0)
    return df

df = flag_high_values(df, m_series_columns, threshold=1000)


In [19]:
# 5. Flagging Non-Metal Related Features
df['M40_NON_METAL_flag'] = np.where(df['98. 6.2 - M40 NON-METAL'] > 0, 1, 0)
df['M61_NON_METAL_flag'] = np.where(df['101. 6.2 - M61 NON-METAL'] > 0, 1, 0)

# 6. Flagging Off-Site Release (when total exceeds a threshold, e.g., 1000)
df['Off_Site_Release_High_flag'] = np.where(df['Total Off-Site Releases'] > 1000, 1, 0)

# 7. Flagging Production Waste (production waste greater than a threshold, e.g., 5000)
df['Production_Waste_High_flag'] = np.where(df['119. PRODUCTION WSTE (8.1-8.7)'] > 5000, 1, 0)


In [20]:
# Mean, Standard Deviation, Median of Waste and Emissions
df['Mean Waste'] = df[waste_columns].mean(axis=1, skipna=True)
df['Mean Emissions'] = df[emission_columns].mean(axis=1, skipna=True)

df['Std Waste'] = df[waste_columns].std(axis=1, skipna=True)
df['Std Emissions'] = df[emission_columns].std(axis=1, skipna=True)

df['Median Waste'] = df[waste_columns].median(axis=1, skipna=True)
df['Median Emissions'] = df[emission_columns].median(axis=1, skipna=True)

# Binned Features for Waste and Emissions
df['Waste Bin'] = pd.cut(df['Total Waste'], bins=[-np.inf, 1000, 5000, np.inf], labels=['Low', 'Medium', 'High'])
df['Emissions Bin'] = pd.cut(df['Total Emissions'], bins=[-np.inf, 500, 2000, np.inf], labels=['Low', 'Medium', 'High'])

# Cumulative Sum Features for Waste and Emissions
df['Cumulative Waste'] = df['Total Waste'].cumsum()
df['Cumulative Emissions'] = df['Total Emissions'].cumsum()

# Interaction Feature: Risk Factor and Total Waste
df['Risk Factor x Total Waste'] = df['Risk Factor'] * df['Total Waste']

# Waste to Emissions Ratio
df['Waste to Emissions Ratio'] = df['Total Waste'] / (df['Total Emissions'] + 1)

In [21]:
import pandas as pd
import numpy as np

# List of chemicals
chemicals = np.array([
    'Hydrochloric acid (acid aerosols including mists, vapors, gas, fog, and other airborne forms of any particle size)',
    'Vanadium compounds', 'p-Cresol', 'Zinc compounds', 'Nonylphenol', 'Manganese compounds',
    'Lead', 'Xylene (mixed isomers)', 'Polycyclic aromatic compounds', 'Nitric acid', 'Xylene (mixed isomers)',
    'Pentachloroethane', 'Phenol', 'Lead', 'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'Manganese compounds', 'Methanol', 'Lead compounds', 'Dimethylamine', 'Dioxin and dioxin-like compounds',
    'Nitric acid', 'Diisocyanates', 'Toluene', 'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'Nonylphenol Ethoxylates', 'Lead compounds', 'Mercury  And Mercury Compounds', 'Methanol', 'Diisocyanates',
    'Lead', 'Nickel', 'Maleic anhydride', 'Hydrogen sulfide', 'Ammonia', 'Chlorophenols', 'Zinc compounds',
    'Copper compounds', 'Manganese  And Manganese Compounds', 'Manganese', 'Mercury  And Mercury Compounds',
    'Copper  And Copper Compounds', 'Ethylene glycol', 'Copper', 'n-Hexane', 'Toluene',
    'Lead  And Lead Compounds', 'Phthalic anhydride', 'Nickel', 'Nitric acid', 'Diisocyanates',
    'Methyl isobutyl ketone', 'Lead  And Lead Compounds', 'Lead', 'Cobalt', 'n-Hexane', 'Copper',
    'Nitrate compounds (water dissociable; reportable only when in aqueous solution)', 'Certain glycol ethers',
    'Mercury compounds', 'Nickel', 'Copper', 'Chromium  and Chromium Compounds(except for chromite ore mined in the Transvaal Region)',
    'Copper', 'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'Formaldehyde', 'Copper', 'Manganese', 'Manganese  And Manganese Compounds', 'Hexachlorocyclopentadiene',
    'Certain glycol ethers', 'Biphenyl', '1,2-Butylene oxide', 'Ethyl acrylate', 'Methanol', 'Nickel',
    'Sulfuric acid (acid aerosols including mists, vapors, gas, fog, and other airborne forms of any particle size)',
    'Methanol', 'Pentobarbital sodium', 'Dichloromethane', 'Lead', 'n-Hexane', 'Hydrogen cyanide', 'Lead',
    'Hydrogen fluoride', 'Xylene (mixed isomers)', 'Dioxin and dioxin-like compounds', 'Chromium',
    'Certain glycol ethers', 'Ammonia', 'Dicyclopentadiene', 'Benzo[g,h,i]perylene', 'Methanol', 'Ammonia',
    'Ammonia', 'Manganese  And Manganese Compounds', 'Ethylbenzene', 'Methanol', 'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'n-Hexane', 'Manganese', 'Dioxin and dioxin-like compounds', 'Styrene', 'Lead', 'Aldrin', 'Chromium',
    'Nickel', 'Polycyclic aromatic compounds', 'Manganese', 'Zinc compounds', 'Arsenic compounds', 'Naphthalene',
    'Diethanolamine', 'Molybdenum trioxide', 'Zinc compounds', 'Lead  And Lead Compounds', 'Acetaldehyde',
    'Manganese compounds', 'Toluene', 'Dioxin and dioxin-like compounds', 'Toluene', 'Certain glycol ethers',
    'Diisocyanates', 'Zinc (fume or dust)', 'Arsenic', 'n-Hexane', '1,2,4-Trimethylbenzene', 'Polycyclic aromatic compounds',
    'Formaldehyde', 'Benzene', 'Nonylphenol Ethoxylates', 'Styrene', 'Manganese', 'Chromium', 'Methanol',
    'Polycyclic aromatic compounds', 'Chromium', 'Mercury  And Mercury Compounds', 'Arsenic compounds', 'Manganese',
    'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'Chromium  and Chromium Compounds(except for chromite ore mined in the Transvaal Region)',
    'Polycyclic aromatic compounds', 'Copper', 'Barium compounds (except for barium sulfate (CAS No. 7727-43-7))',
    'Chromium', 'Toluene', 'Ammonia', 'n-Butyl alcohol', 'Nitrate compounds (water dissociable; reportable only when in aqueous solution)',
    'Toluene', 'Methanol'
])

# Define categories for grouping
chemical_categories = {
    'Heavy Metals': ['Lead', 'Mercury', 'Copper', 'Zinc', 'Nickel', 'Arsenic', 'Manganese', 'Chromium', 'Vanadium', 'Cobalt'],
    'Acids': ['Hydrochloric acid', 'Nitric acid', 'Sulfuric acid', 'Acetaldehyde', 'Methanol'],
    'Aromatic Compounds': ['Xylene', 'Styrene', 'Ethylbenzene', 'Toluene', 'Naphthalene', 'Polycyclic aromatic compounds'],
    'Other Organic Compounds': ['Methanol', 'Dimethylamine', 'Formaldehyde', 'Diisocyanates'],
    'Industrial Chemicals': ['Nonylphenol', 'Nitrate compounds', 'Biphenyl', 'Ethyl acrylate', 'Dichloromethane', 'Maleic anhydride'],
    'Glycol Ethers': ['Certain glycol ethers'],
    'Pesticides': ['Aldrin']
}

# Create a function to categorize chemicals
def categorize_chemical(chemical_name):
    for category, chemicals in chemical_categories.items():
        for chem in chemicals:
            if chem in chemical_name:
                return category
    return 'Other'  # If not found in any category



In [22]:
# Define categories for grouping
chemical_categories = {
    'Heavy Metals': ['Lead', 'Mercury', 'Copper', 'Zinc', 'Nickel', 'Arsenic', 'Manganese', 'Chromium', 'Vanadium', 'Cobalt'],
    'Acids': ['Hydrochloric acid', 'Nitric acid', 'Sulfuric acid', 'Acetaldehyde', 'Methanol'],
    'Aromatic Compounds': ['Xylene', 'Styrene', 'Ethylbenzene', 'Toluene', 'Naphthalene', 'Polycyclic aromatic compounds'],
    'Other Organic Compounds': ['Methanol', 'Dimethylamine', 'Formaldehyde', 'Diisocyanates'],
    'Industrial Chemicals': ['Nonylphenol', 'Nitrate compounds', 'Biphenyl', 'Ethyl acrylate', 'Dichloromethane', 'Maleic anhydride'],
    'Glycol Ethers': ['Certain glycol ethers'],
    'Pesticides': ['Aldrin']
}

# Create a function to categorize chemicals
def categorize_chemical(chemical_name):
    for category, chemicals in chemical_categories.items():
        for chem in chemicals:
            if chem in chemical_name:
                return category
    return 'Other'  # If not found in any category

# Apply categorization and create a new 'Category' column
df['Category'] = df['37. CHEMICAL'].apply(categorize_chemical)

In [ ]:
def classify_risk_and_hazard(chemical_name):
    # Define high, medium, and low risk chemicals based on hazardous nature
    high_risk = ['Lead', 'Mercury', 'Arsenic', 'Chromium', 'Dioxin', 'Aldrin']
    medium_risk = ['Xylene', 'Toluene', 'Manganese', 'Nickel', 'Zinc']
    low_risk = ['Methanol', 'Formaldehyde', 'Ethylbenzene', 'Copper']

    hazard_types = {
        'Toxic': ['Lead', 'Mercury', 'Arsenic'],
        'Corrosive': ['Hydrochloric acid', 'Sulfuric acid', 'Nitric acid'],
        'Flammable': ['Xylene', 'Toluene', 'Ethylbenzene'],
        'Environmental': ['Pesticides', 'Nonylphenol']
    }

    # Classify risk
    risk_level = 'Low'
    for chem in high_risk:
        if chem in chemical_name:
            risk_level = 'High'
            break
    for chem in medium_risk:
        if chem in chemical_name:
            risk_level = 'Medium'
            break

    # Classify hazard
    hazard = 'None'
    for hazard_type, chemicals in hazard_types.items():
        for chem in chemicals:
            if chem in chemical_name:
                hazard = hazard_type
                break

    return risk_level, hazard

# Apply the function to classify risk and hazard
df[['Risk_Level', 'Hazard_Type']] = df['37. CHEMICAL'].apply(lambda x: pd.Series(classify_risk_and_hazard(x)))

# Feature Engineering for production waste prediction (example features)
df['CHEMICAL_Category'] = df['37. CHEMICAL'].apply(categorize_chemical)
df['Risk_Score'] = df['Risk_Level'].apply(lambda x: 3 if x == 'High' else (2 if x == 'Medium' else 1))
df['Hazard_Score'] = df['Hazard_Type'].apply(lambda x: 3 if x == 'Toxic' else (2 if x == 'Corrosive' else 1))


In [ ]:
import pandas as pd

def detect_outliers(df, columns):
    """
    Function to detect outliers in specified columns of a dataframe using the IQR method.
    The function returns the indices of outliers.
    """
    outlier_indices = set()  # Using a set to avoid duplicate indices

    # Loop through the specified columns in the dataframe
    for col in columns:
        if col in df.columns:
            # Check if the column is numeric (either int or float)
            if pd.api.types.is_numeric_dtype(df[col]):
                # Check if the column has at least two unique values (avoid constant columns)
                if df[col].nunique() > 1:
                    # Drop rows with NaN values in the column (or use fillna if preferred)
                    df_col = df[col].dropna()

                    # Check if the column is not empty after dropping NaNs
                    if len(df_col) == 0:
                        continue  # Skip empty columns

                    Q1 = df_col.quantile(0.25)  # First quartile (25%)
                    Q3 = df_col.quantile(0.75)  # Third quartile (75%)
                    IQR = Q3 - Q1  # Interquartile range

                    # Handle division by zero if IQR is zero
                    if IQR == 0:
                        continue  # No outliers if IQR is zero

                    # Define the lower and upper bounds for outliers
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR

                    # Detect outliers (values outside the bounds)
                    outlier = df[col][(df[col] < lower_bound) | (df[col] > upper_bound)]

                    # Add the indices of the outliers to the set
                    outlier_indices.update(outlier.index)

                    # Print the number of outliers in the column
                    print(f"Number of outliers in {col}: {outlier.count()}")
            else:
                print(f"Skipping column {col} because it is not numeric.")

    return outlier_indices

# List of columns you want to check for outliers
columns_to_check = [
    '22. INDUSTRY SECTOR CODE', '51. 5.1 - FUGITIVE AIR', '52. 5.2 - STACK AIR', '53. 5.3 - WATER',
    '55. 5.4.1 - UNDERGROUND CL I', '56. 5.4.2 - UNDERGROUND C II-V', '58. 5.5.1A - RCRA C LANDFILL',
    '59. 5.5.1B - OTHER LANDFILLS', '60. 5.5.2 - LAND TREATMENT', '62. 5.5.3A - RCRA SURFACE IM',
    '63. 5.5.3B - OTHER SURFACE I', '64. 5.5.4 - OTHER DISPOSAL', '65. ON-SITE RELEASE TOTAL',
    '66. 6.1 - POTW - TRNS RLSE', '67. 6.1 - POTW - TRNS TRT', '68. POTW - TOTAL TRANSFERS', '69. 6.2 - M10',
    '70. 6.2 - M41', '71. 6.2 - M62', '75. 6.2 - M81', '76. 6.2 - M82', '79. 6.2 - M66', '80. 6.2 - M67',
    '81. 6.2 - M64', '82. 6.2 - M65', '83. 6.2 - M73', '84. 6.2 - M79', '85. 6.2 - M90', '86. 6.2 - M94',
    '87. 6.2 - M99', '88. OFF-SITE RELEASE TOTAL', '89. 6.2 - M20', '90. 6.2 - M24', '91. 6.2 - M26',
    '92. 6.2 - M28', '93. 6.2 - M93', '94. OFF-SITE RECYCLED TOTAL', '95. 6.2 - M56', '96. 6.2 - M92',
    '97. OFF-SITE ENERGY RECOVERY T', '98. 6.2 - M40 NON-METAL', '99. 6.2 - M50', '100. 6.2 - M54',
    '101. 6.2 - M61 NON-METAL', '102. 6.2 - M69', '103. 6.2 - M95', '104. OFF-SITE TREATED TOTAL',
    '106. 6.2 - TOTAL TRANSFER', '107. TOTAL RELEASES', '109. 8.1A - ON-SITE CONTAINED',
    '110. 8.1B - ON-SITE OTHER', '111. 8.1C - OFF-SITE CONTAIN', '112. 8.1D - OFF-SITE OTHER R',
    '113. 8.2 - ENERGY RECOVER ON', '114. 8.3 - ENERGY RECOVER OF', '115. 8.4 - RECYCLING ON SITE',
    '116. 8.5 - RECYCLING OFF SIT', '117. 8.6 - TREATMENT ON SITE', '118. 8.7 - TREATMENT OFF SITE',
    '119. PRODUCTION WSTE (8.1-8.7)', '122. 8.9 - PRODUCTION RATIO', '121. PROD_RATIO_OR_ ACTIVITY',
    '21. FEDERAL FACILITY', '37. CHEMICAL', '38. ELEMENTAL METAL INCLUDED', '42. CLEAN AIR ACT CHEMICAL',
    '43. CLASSIFICATION', '44. METAL', '45. METAL CATEGORY', '46. CARCINOGEN', '47. PBT', '48. PFAS',
    '49. FORM TYPE', '50. UNIT OF MEASURE'
]

# Example usage:
df_final = df.copy()  # Assuming 'df_final' is your DataFrame with data
outliers_indices = detect_outliers(df_final, columns_to_check)

# Now you can filter out the outliers from the DataFrame
df_no_outliers = df_final.drop(index=outliers_indices)

print(f"Filtered DataFrame without outliers: \n{df_no_outliers}")


In [ ]:
len(df_no_outliers)

In [ ]:
df['37. CHEMICAL'].values

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# Assuming df_no_outliers is already preprocessed
df_final = df_no_outliers.copy()

# Step 1: Convert all categorical columns to string type
categorical_cols = df_final.select_dtypes(include=['object','bool']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Apply Label Encoding to all categorical columns
label_encoder = LabelEncoder()

for col in X.select_dtypes(include=['object','category']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Step 4: Apply Standard Scaling to the features
scaler = StandardScaler()

# Only scale the numeric columns (exclude the target and categorical columns)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Step 6: Initialize and train the GradientBoostingRegressor model
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# X_train=X_train.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# X_test=X_test.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
import xgboost as xgb

# Step 6: Initialize and train the XGBoost model
model = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
# Step 6: Initialize and train the CatBoostRegressor model
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.1, loss_function='RMSE',  random_state=42, verbose=100)

# Train the model
model.fit(X_train, y_train)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')

In [ ]:
# Step 6: Initialize and train the LGBMRegressor model
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the LGBMRegressor model
model = LGBMRegressor(n_estimators=1000,
                      max_depth=10,
                      learning_rate=0.1,
                      objective='regression',
                      random_state=42,
                      verbose=100)

# Train the model
model.fit(X_train, y_train)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Step 6: Initialize and train the GradientBoostingRegressor model
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# X_train=X_train.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# X_test=X_test.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
len(df_no_outliers)

# WITH FE

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# Assuming df_no_outliers is already preprocessed
df_final = df_no_outliers.copy()

# Step 1: Convert all categorical columns to string type
categorical_cols = df_final.select_dtypes(include=['object','bool']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Apply Label Encoding to all categorical columns
label_encoder = LabelEncoder()

for col in X.select_dtypes(include=['object','category']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Step 4: Apply Standard Scaling to the features
scaler = StandardScaler()

# Only scale the numeric columns (exclude the target and categorical columns)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Step 6: Initialize and train the GradientBoostingRegressor model
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# X_train=X_train.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# X_test=X_test.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
import xgboost as xgb

# Step 6: Initialize and train the XGBoost model
model = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
import xgboost as xgb
import numpy as np
from pyswarm import pso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# Assume the following variables are already set up:
# df_final, target_column, X, y, X_train, X_test, y_train, y_test

# Define the objective function for PSO
def objective_function(params):
    learning_rate, max_depth, n_estimators, subsample, colsample_bytree = params

    # Train XGBoost model with the given parameters
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=int(max_depth),  # max_depth should be an integer
        n_estimators=int(n_estimators),  # n_estimators should be an integer
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    return mae  # We want to minimize MAE

# Define the boundaries for the hyperparameters
# Bounds: [learning_rate, max_depth, n_estimators, subsample, colsample_bytree]
lb = [0.01, 3, 50, 0.5, 0.5]  # Lower bounds of the parameters
ub = [0.1, 8, 100, 0.9, 0.9]  # Upper bounds of the parameters (narrowed range)

# Perform PSO optimization with minimal swarm size and iterations
best_params, _ = pso(objective_function, lb, ub, swarmsize=5, maxiter=10, debug=True)

# Print the best parameters found by PSO
print(f"Best parameters found by PSO: {best_params}")

# Extract the best parameters
learning_rate, max_depth, n_estimators, subsample, colsample_bytree = best_params

# Train the final XGBoost model using the optimized parameters
final_model = xgb.XGBRegressor(
    learning_rate=learning_rate,
    max_depth=int(max_depth),
    n_estimators=int(n_estimators),
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    random_state=42
)

# Fit the final model
final_model.fit(X_train, y_train)

# Make predictions using the final model
y_pred_final = final_model.predict(X_test)

# Evaluate the final model
final_mae = mean_absolute_error(y_test, y_pred_final)
final_mse = mean_squared_error(y_test, y_pred_final)
final_r2 = r2_score(y_test, y_pred_final)

# Print evaluation metrics
print(f"Final Model - Mean Absolute Error (MAE): {final_mae}")
print(f"Final Model - Mean Squared Error (MSE): {final_mse}")
print(f"Final Model - R2 Score: {final_r2}")


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# Assume the following variables are already set up:
# df_final, target_column, X, y, X_train, X_test, y_train, y_test

# Define the objective function for ACO
def objective_function(params):
    learning_rate, max_depth, n_estimators, subsample, colsample_bytree = params

    # Train XGBoost model with the given parameters
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=int(max_depth),  # max_depth should be an integer
        n_estimators=int(n_estimators),  # n_estimators should be an integer
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    return mae  # We want to minimize MAE

# Simple ACO Hyperparameter Optimization
class AntColonyOptimizer:
    def __init__(self, objective_function, bounds, n_ants=5, n_iter=10, alpha=1, beta=1, rho=0.9):
        self.objective_function = objective_function
        self.bounds = bounds
        self.n_ants = n_ants
        self.n_iter = n_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho

        # Initialize pheromone levels (same for all dimensions)
        self.pheromone = np.ones((len(bounds), 10))  # 10 is an arbitrary precision size
        self.best_score = float('inf')
        self.best_solution = None

    def optimize(self):
        for iteration in range(self.n_iter):
            solutions = []
            scores = []
            for _ in range(self.n_ants):
                solution = self.construct_solution()
                score = self.objective_function(solution)
                solutions.append(solution)
                scores.append(score)
                if score < self.best_score:
                    self.best_score = score
                    self.best_solution = solution
            self.update_pheromone(solutions, scores)
        return self.best_solution, self.best_score

    def construct_solution(self):
        solution = []
        for i, (lb, ub) in enumerate(self.bounds):
            prob = self.pheromone[i] ** self.alpha
            prob /= np.sum(prob)  # Normalize the probabilities

            # Pick the index of the best probability based on pheromone level
            index = np.random.choice(range(10), p=prob)
            value = lb + (index / 9.0) * (ub - lb)  # Scale it to the actual bounds
            solution.append(value)
        return solution

    def update_pheromone(self, solutions, scores):
        # Update pheromones based on the best solutions
        delta_pheromone = np.zeros_like(self.pheromone)
        for solution, score in zip(solutions, scores):
            for i, value in enumerate(solution):
                delta_pheromone[i] += (self.best_score - score) / self.best_score  # Reward better solutions

        # Apply pheromone evaporation
        self.pheromone *= self.rho
        # Add new pheromone
        self.pheromone += delta_pheromone

# Define the bounds for the hyperparameters
# Bounds: [learning_rate, max_depth, n_estimators, subsample, colsample_bytree]
lb = [0.01, 3, 50, 0.5, 0.5]  # Lower bounds of the parameters
ub = [0.1, 8, 100, 0.9, 0.9]  # Upper bounds of the parameters (narrowed range)

# Perform ACO optimization with minimal swarm size and iterations
aco_optimizer = AntColonyOptimizer(objective_function, bounds=list(zip(lb, ub)), n_ants=5, n_iter=10)
best_params, best_score = aco_optimizer.optimize()

# Print the best parameters found by ACO
print(f"Best parameters found by ACO: {best_params}")

# Extract the best parameters
learning_rate, max_depth, n_estimators, subsample, colsample_bytree = best_params

# Train the final XGBoost model using the optimized parameters
final_model = xgb.XGBRegressor(
    learning_rate=learning_rate,
    max_depth=int(max_depth),
    n_estimators=int(n_estimators),
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    random_state=42
)

# Fit the final model
final_model.fit(X_train, y_train)

# Make predictions using the final model
y_pred_final = final_model.predict(X_test)

# Evaluate the final model
final_mae = mean_absolute_error(y_test, y_pred_final)
final_mse = mean_squared_error(y_test, y_pred_final)
final_r2 = r2_score(y_test, y_pred_final)

# Print evaluation metrics
print(f"Final Model - Mean Absolute Error (MAE): {final_mae}")
print(f"Final Model - Mean Squared Error (MSE): {final_mse}")
print(f"Final Model - R2 Score: {final_r2}")


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# Assume the following variables are already set up:
# df_final, target_column, X, y, X_train, X_test, y_train, y_test

# Define the objective function for Hill Climbing
def objective_function(params):
    learning_rate, max_depth, n_estimators, subsample, colsample_bytree = params

    # Train XGBoost model with the given parameters
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=int(max_depth),  # max_depth should be an integer
        n_estimators=int(n_estimators),  # n_estimators should be an integer
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    return mae  # We want to minimize MAE

# Hill Climbing Hyperparameter Optimization
def hill_climbing(objective_function, bounds, max_iterations=5, step_size=0.01):
    # Generate an initial random solution within bounds
    current_solution = [np.random.uniform(lb, ub) for lb, ub in bounds]
    current_score = objective_function(current_solution)

    best_solution = current_solution
    best_score = current_score

    for _ in range(max_iterations):
        # Generate neighbors by slightly modifying the current solution
        neighbors = []
        for i, (lb, ub) in enumerate(bounds):
            neighbor = current_solution.copy()
            change = np.random.uniform(-step_size, step_size)
            neighbor[i] = np.clip(neighbor[i] + change, lb, ub)  # Ensure values are within bounds
            neighbors.append(neighbor)

        # Evaluate the neighbors and select the best one
        for neighbor in neighbors:
            score = objective_function(neighbor)
            if score < best_score:
                best_score = score
                best_solution = neighbor

        # Update the current solution
        current_solution = best_solution

    return best_solution, best_score

# Define the bounds for the hyperparameters
# Bounds: [learning_rate, max_depth, n_estimators, subsample, colsample_bytree]
lb = [0.01, 3, 50, 0.5, 0.5]  # Lower bounds of the parameters
ub = [0.1, 8, 100, 0.9, 0.9]  # Upper bounds of the parameters (narrowed range)

# Perform Hill Climbing optimization with minimal iterations (5 iterations)
bounds = list(zip(lb, ub))
best_params, best_score = hill_climbing(objective_function, bounds, max_iterations=5)

# Print the best parameters found by Hill Climbing
print(f"Best parameters found by Hill Climbing: {best_params}")

# Extract the best parameters
learning_rate, max_depth, n_estimators, subsample, colsample_bytree = best_params

# Train the final XGBoost model using the optimized parameters
final_model = xgb.XGBRegressor(
    learning_rate=learning_rate,
    max_depth=int(max_depth),
    n_estimators=int(n_estimators),
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    random_state=42
)

# Fit the final model
final_model.fit(X_train, y_train)

# Make predictions using the final model
y_pred_final = final_model.predict(X_test)

# Evaluate the final model
final_mae = mean_absolute_error(y_test, y_pred_final)
final_mse = mean_squared_error(y_test, y_pred_final)
final_r2 = r2_score(y_test, y_pred_final)

# Print evaluation metrics
print(f"Final Model - Mean Absolute Error (MAE): {final_mae}")
print(f"Final Model - Mean Squared Error (MSE): {final_mse}")
print(f"Final Model - R2 Score: {final_r2}")


In [ ]:
pip install pyswarm

In [ ]:
# Step 6: Initialize and train the CatBoostRegressor model
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.1, loss_function='RMSE',  random_state=42, verbose=100)

# Train the model
model.fit(X_train, y_train)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')

In [ ]:
# Step 6: Initialize and train the LGBMRegressor model
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the LGBMRegressor model
model = LGBMRegressor(n_estimators=1000,
                      max_depth=10,
                      learning_rate=0.1,
                      objective='regression',
                      random_state=42,
                      verbose=100)

# Train the model
model.fit(X_train, y_train)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import pandas as pd

df_final = df_no_outliers.copy()

# Step 1: Convert all categorical columns to string type
categorical_cols = df_final.select_dtypes(include=['object','bool']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Apply Label Encoding to all categorical columns
label_encoder = LabelEncoder()

for col in X.select_dtypes(include=['object','category']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Step 4: Apply Standard Scaling to the features
scaler = StandardScaler()

# Only scale the numeric columns (exclude the target and categorical columns)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
xgb_model = XGBRegressor(n_estimators=100)
gbr_model = GradientBoostingRegressor(n_estimators=100,)
rf_model = RandomForestRegressor(n_estimators=100)

# Step 6: Train the base models
xgb_model.fit(X_train, y_train)
gbr_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# Step 7: Make predictions from the base models on the training and test set
xgb_train_pred = xgb_model.predict(X_train)
gbr_train_pred = gbr_model.predict(X_train)
rf_train_pred = rf_model.predict(X_train)

xgb_test_pred = xgb_model.predict(X_test)
gbr_test_pred = gbr_model.predict(X_test)
rf_test_pred = rf_model.predict(X_test)

# Step 8: Stack the predictions (use the predictions as features for the meta-model)
stacked_train_predictions = np.column_stack((xgb_train_pred ,rf_train_pred))
stacked_test_predictions = np.column_stack((xgb_test_pred, rf_test_pred))

# Step 9: Define the meta-model (GradientBoostingRegressor)
meta_model = GradientBoostingRegressor(n_estimators=100)

# Step 10: Train the meta-model using the stacked predictions
meta_model.fit(stacked_train_predictions, y_train)

# Step 11: Make predictions using the meta-model on the test set
final_predictions = meta_model.predict(stacked_test_predictions)

# Step 12: Calculate RMSE and MAE for the training and test set
train_rmse = np.sqrt(mean_squared_error(y_train, meta_model.predict(stacked_train_predictions)))
train_mae = mean_absolute_error(y_train, meta_model.predict(stacked_train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
test_mae = mean_absolute_error(y_test, final_predictions)

print(f'Training RMSE: {train_rmse}')
print(f'Training MAE: {train_mae}')
print(f'Test RMSE: {test_rmse}')
print(f'Test MAE: {test_mae}')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Define models with weights (XGBoost, GradientBoosting, RandomForest)
models_with_weights = {
    'XGBRegressor': (XGBRegressor(n_estimators=100), 0.1),
    'GradientBoostingRegressor': (GradientBoostingRegressor(n_estimators=100), 0.8),
    'RandomForestRegressor': (RandomForestRegressor(n_estimators=100), 0.1)
}

# Initialize DataFrame for storing predictions
blended_train_pred = pd.DataFrame()
blended_test_pred = pd.DataFrame()

# Loop over models and make predictions
for model_name, (model, weight) in models_with_weights.items():
    # Fit model and generate predictions
    model.fit(X_train, y_train)

    # Flatten predictions and apply weight
    blended_train_pred[model_name] = weight * model.predict(X_train).squeeze()
    blended_test_pred[model_name] = weight * model.predict(X_test).squeeze()

print(f'blended_train_pred.shape: {blended_train_pred.shape}')
print(f'blended_test_pred.shape: {blended_test_pred.shape}')

# Calculate RMSE for the training data
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

train_rmse = rmse(y_train, blended_train_pred.sum(axis='columns'))
print(f'Training RMSE: {train_rmse}')

# Calculate MAE for the training data
train_mae = mean_absolute_error(y_train, blended_train_pred.sum(axis='columns'))
print(f'Training MAE: {train_mae}')

# Generate final test predictions
test_pred = np.floor(np.expm1(blended_test_pred.sum(axis='columns')))

# Calculate RMSE for test predictions
test_rmse = rmse(y_test, blended_test_pred.sum(axis='columns'))
print(f'Test RMSE: {test_rmse}')

# Calculate MAE for test predictions
test_mae = mean_absolute_error(y_test, blended_test_pred.sum(axis='columns'))
print(f'Test MAE: {test_mae}')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Define models with weights (XGBoost, GradientBoosting, RandomForest)
models_with_weights = {
    'XGBRegressor': (XGBRegressor(n_estimators=100), 0.1),
    'GradientBoostingRegressor': (GradientBoostingRegressor(n_estimators=100), 0.8),
    'RandomForestRegressor': (RandomForestRegressor(n_estimators=100), 0.1)
}

# Initialize DataFrame for storing predictions
blended_train_pred = pd.DataFrame()
blended_test_pred = pd.DataFrame()

# Loop over models and make predictions
for model_name, (model, weight) in models_with_weights.items():
    # Fit model and generate predictions
    model.fit(X_train, y_train)

    # Flatten predictions and apply weight
    blended_train_pred[model_name] = weight * model.predict(X_train).squeeze()
    blended_test_pred[model_name] = weight * model.predict(X_test).squeeze()

print(f'blended_train_pred.shape: {blended_train_pred.shape}')
print(f'blended_test_pred.shape: {blended_test_pred.shape}')

# Calculate RMSE for the training data
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

train_rmse = rmse(y_train, blended_train_pred.sum(axis='columns'))
print(f'Training RMSE: {train_rmse}')

# Calculate MAE for the training data
train_mae = mean_absolute_error(y_train, blended_train_pred.sum(axis='columns'))
print(f'Training MAE: {train_mae}')

# Generate final test predictions
test_pred = np.floor(np.expm1(blended_test_pred.sum(axis='columns')))

# Calculate RMSE for test predictions
test_rmse = rmse(y_test, blended_test_pred.sum(axis='columns'))
print(f'Test RMSE: {test_rmse}')

# Calculate MAE for test predictions
test_mae = mean_absolute_error(y_test, blended_test_pred.sum(axis='columns'))
print(f'Test MAE: {test_mae}')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# Assuming df_no_outliers is already preprocessed
df_final = df_no_outliers.copy()

# Step 1: Convert all categorical columns to string type
categorical_cols = df_final.select_dtypes(include=['object','bool']).columns
df_final[categorical_cols] = df_final[categorical_cols].astype(str)

# Step 2: Define the target and features
target_column = '119. PRODUCTION WSTE (8.1-8.7)'  # Assuming this is the target column
X = df_final.drop(columns=[target_column])  # Features (excluding target)
y = df_final[target_column]  # Target (Production Waste)

# Step 3: Apply Label Encoding to all categorical columns
label_encoder = LabelEncoder()

for col in X.select_dtypes(include=['object','category']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Step 4: Apply Standard Scaling to the features
scaler = StandardScaler()

# Only scale the numeric columns (exclude the target and categorical columns)
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
df_final.info()

In [ ]:
df_final['119. PRODUCTION WSTE (8.1-8.7)'].mean()

In [ ]:
np.sqrt(mse)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Step 6: Initialize and train the GradientBoostingRegressor model
model = GradientBoostingRegressor(n_estimators=100, random_state=4)
# X_train=X_train.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# X_test=X_test.drop(['Total Waste to Facility Ratio','Total Emissions to Facility Ratio'],axis=1)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
import xgboost as xgb

# Step 6: Initialize and train the XGBoost model
model = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.linear_model import LinearRegression

# Step 6: Initialize and train the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
from sklearn.svm import SVR

# Step 6: Initialize and train the Support Vector Regressor model
model = SVR()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')


In [ ]:
# Step 6: Initialize and train the CatBoostRegressor model
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.1, loss_function='RMSE',  random_state=42, verbose=100)

# Train the model
model.fit(X_train, y_train)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R2 Score: {r2}')

In [ ]:
from pyswarm import pso
import xgboost as xgb


def objective_function(params):
    """
    Fungsi objektif yang digunakan oleh PSO untuk mengevaluasi model XGBoost.
    params[0]: learning_rate
    params[1]: max_depth
    params[2]: n_estimators
    params[3]: subsample
    """
    learning_rate = params[0]
    max_depth = int(params[1])
    n_estimators = int(params[2])
    subsample = params[3]

    # Inisialisasi model XGBoost
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=max_depth,
        n_estimators=n_estimators,
        subsample=subsample,
        objective='reg:squarederror',
        random_state=42,
        verbosity=0
    )

    # Latih model
    model.fit(X_train, y_train)

    # Prediksi pada data test
    y_pred = model.predict(X_test)

    # Hitung metrik MAE (Mean Absolute Error)
    mae = mean_absolute_error(y_test, y_pred)
    return mae  # Kita optimalkan MAE yang lebih rendah

# Langkah 6: Tentukan batas untuk parameter PSO
lb = [0.01, 3, 50, 0.6]  # Batas bawah [learning_rate, max_depth, n_estimators, subsample]
ub = [0.2, 15, 500, 1.0]  # Batas atas [learning_rate, max_depth, n_estimators, subsample]

# Langkah 7: Terapkan PSO untuk menemukan parameter terbaik
best_params, best_obj_value = pso(objective_function, lb, ub, swarmsize=10, maxiter=5)

# Menampilkan parameter terbaik yang ditemukan oleh PSO
print(f"Best Parameters: Learning Rate={best_params[0]}, Max Depth={int(best_params[1])}, N Estimators={int(best_params[2])}, Subsample={best_params[3]}")
print(f"Best Objective Value (MAE): {best_obj_value}")

# Langkah 8: Gunakan parameter terbaik untuk melatih model akhir
best_learning_rate = best_params[0]
best_max_depth = int(best_params[1])
best_n_estimators = int(best_params[2])
best_subsample = best_params[3]

final_model = xgb.XGBRegressor(
    learning_rate=best_learning_rate,
    max_depth=best_max_depth,
    n_estimators=best_n_estimators,
    subsample=best_subsample,
    objective='reg:squarederror',
    random_state=42,
    verbosity=0
)

# Latih model dengan parameter terbaik
final_model.fit(X_train, y_train)

# Prediksi dengan model terbaik
y_pred_final = final_model.predict(X_test)

# Evaluasi model akhir
mae = mean_absolute_error(y_test, y_pred_final)
mse = mean_squared_error(y_test, y_pred_final)
r2 = r2_score(y_test, y_pred_final)

# Metrik evaluasi
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")

In [ ]:
def objective_function(params):
    """
    Fungsi objektif yang digunakan untuk mengevaluasi model XGBoost dengan parameter tertentu.
    params[0]: learning_rate
    params[1]: max_depth
    params[2]: n_estimators
    params[3]: subsample
    """
    learning_rate = params[0]
    max_depth = int(params[1])
    n_estimators = int(params[2])
    subsample = params[3]

    # Inisialisasi model XGBoost
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=max_depth,
        n_estimators=n_estimators,
        subsample=subsample,
        objective='reg:squarederror',
        random_state=42,
        verbosity=0
    )

    # Latih model
    model.fit(X_train, y_train)

    # Prediksi pada data test
    y_pred = model.predict(X_test)

    # Hitung metrik MAE (Mean Absolute Error)
    mae = mean_absolute_error(y_test, y_pred)
    return mae  # Kita optimalkan MAE yang lebih rendah

# Langkah 6: Inisialisasi parameter acuan untuk Hill Climbing
current_params = [0.1, 6, 100, 0.8]  # Inisialisasi dengan beberapa nilai parameter awal
current_score = objective_function(current_params)

# Langkah 7: Hill Climbing Loop
def hill_climb():
    global current_params, current_score
    step_size = 0.01  # Langkah perubahan untuk setiap parameter
    max_iterations = 100  # Maksimal iterasi pencarian

    for iteration in range(max_iterations):
        # Generate new parameters by making small changes to the current ones
        new_params = current_params.copy()

        # Cobalah perubahan kecil untuk setiap parameter
        new_params[0] += np.random.uniform(-step_size, step_size)  # learning_rate
        new_params[1] = int(new_params[1] + np.random.choice([-1, 1]) * np.random.randint(1, 3))  # max_depth
        new_params[2] = int(new_params[2] + np.random.choice([-1, 1]) * np.random.randint(1, 20)))  # n_estimators
        new_params[3] += np.random.uniform(-step_size, step_size)  # subsample

        # Bound parameters within their limits
        new_params[0] = np.clip(new_params[0], 0.01, 0.2)
        new_params[1] = np.clip(new_params[1], 3, 15)
        new_params[2] = np.clip(new_params[2], 50, 500)
        new_params[3] = np.clip(new_params[3], 0.6, 1.0)

        # Evaluasi parameter baru
        new_score = objective_function(new_params)

        # Jika parameter baru lebih baik, lakukan pembaruan
        if new_score < current_score:
            print(f"Iteration {iteration + 1}: Improved MAE = {new_score} with parameters {new_params}")
            current_params = new_params
            current_score = new_score

# Langkah 8: Jalankan Hill Climbing
hill_climb()

# Langkah 9: Gunakan parameter terbaik yang ditemukan untuk melatih model akhir
best_learning_rate = current_params[0]
best_max_depth = int(current_params[1])
best_n_estimators = int(current_params[2])
best_subsample = current_params[3]

final_model = xgb.XGBRegressor(
    learning_rate=best_learning_rate,
    max_depth=best_max_depth,
    n_estimators=best_n_estimators,
    subsample=best_subsample,
    objective='reg:squarederror',
    random_state=42,
    verbosity=0
)

# Latih model dengan parameter terbaik
final_model.fit(X_train, y_train)

# Prediksi dengan model terbaik
y_pred_final = final_model.predict(X_test)

# Evaluasi model akhir
mae = mean_absolute_error(y_test, y_pred_final)
mse = mean_squared_error(y_test, y_pred_final)
r2 = r2_score(y_test, y_pred_final)

# Metrik evaluasi
print(f"Final Model Mean Absolute Error (MAE): {mae}")
print(f"Final Model Mean Squared Error (MSE): {mse}")
print(f"Final Model R2 Score: {r2}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Extract feature importances
feature_importances = model.feature_importances_
features = X_train.columns  # Assuming X_train is a DataFrame

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Plot the top 10 feature importances
plt.figure(figsize=(10, 6))
top_features = importance_df.head(10)
plt.barh(top_features['Feature'], top_features['Importance'], color='#5038bc')
plt.gca().invert_yaxis()
plt.title('Top 10 Feature Importances', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.savefig("Top 10 Feature_GB")
plt.show()

# Scatter plot for actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6, color='#5038bc')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title('Actual vs Predicted Values', fontsize=16)
plt.xlabel('Actual Values', fontsize=14)
plt.ylabel('Predicted Values', fontsize=14)
plt.grid(True)
plt.savefig("pred")
plt.show()


In [ ]:
import shap

# Initialize SHAP explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Limit to top 5 features based on mean absolute SHAP values
shap_values_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Mean Abs SHAP": np.abs(shap_values.values).mean(axis=0)
}).sort_values(by="Mean Abs SHAP", ascending=False)

top_5_features = shap_values_df.head(5)["Feature"].tolist()

# Filter SHAP values and X_test for the top 5 features
top_5_indices = [X_train.columns.get_loc(feature) for feature in top_5_features]
filtered_shap_values = shap_values[:, top_5_indices]
filtered_X_test = X_test[top_5_features]

# SHAP Summary Plot (Bar)
shap.summary_plot(filtered_shap_values, filtered_X_test, plot_type="bar", color="#5038bc")

# SHAP Beeswarm Plot (Detailed Insights)
shap.summary_plot(filtered_shap_values, filtered_X_test, color="#5038bc")


In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Initialize SHAP explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# Limit to top 5 features based on mean absolute SHAP values
shap_values_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Mean Abs SHAP": np.abs(shap_values.values).mean(axis=0)
}).sort_values(by="Mean Abs SHAP", ascending=False)

top_5_features = shap_values_df.head(10)["Feature"].tolist()

# Filter SHAP values and X_test for the top 5 features
top_5_indices = [X_train.columns.get_loc(feature) for feature in top_5_features]
filtered_shap_values = shap_values[:, top_5_indices]
filtered_X_test = X_test[top_5_features]

# SHAP Beeswarm Plot (Detailed Insights for Top 5 Features)
plt.figure()  # Create a new figure for saving
shap.summary_plot(filtered_shap_values, filtered_X_test, color="#5038bc", show=False)  # Prevent auto-show
plt.savefig("shap_beeswarm_plot.png", bbox_inches="tight", dpi=300)  # Save the plot
plt.show()  # Display the plot


In [ ]:
import shap
import matplotlib.pyplot as plt
plt.figure()  # Create a new figure for saving
# SHAP Waterfall Plot for a specific instance (e.g., the first test sample)
shap.waterfall_plot(
    shap.Explanation(
        values=filtered_shap_values.values[0],
        base_values=explainer.expected_value,
        feature_names=top_5_features
    )
)
plt.savefig("shap_waterfall_plot.png", bbox_inches="tight", dpi=300)  # Save the plot
plt.show()  # Display the plot
